In [2]:
'''
https://scikit-learn.org/stable/modules/permutation_importance.html
'''
import numpy as np
import os
import pyedflib
from utils.EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

import read_edf_files
kernels, chans = 1, 64
label_mapping = {
        1: "Rest",
        2: "Squeeze Both Fists",
        3: "Squeeze Both Feet",
        4: "Squeeze Left Hand",
        5: "Squeeze Right Hand",
    }
num_labels = 5
X, Y = read_edf_files.reader() # use other function to read the edf files


2023-10-02 19:35:51.733627: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 19:35:52.009525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-02 19:35:52.009543: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-02 19:35:52.046914: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 19:35:53.261636: W tensorflow/stream_executor/platform/de

X.shape:  (180, 64, 656)
Y.shape:  (180,)


In [3]:
################################################################
## Process, filter, and epoch the data
# init arrays to train/validate/test. Make split 50/25/25
half = int(len(X) / 2)
quarter = int(half / 2)
three_fourths = half + quarter

X_train = X[:half, :, :]
X_validate = X[half : three_fourths, :, :]
X_test = X[three_fourths:, :, :]

y_train_int = Y[:half]
y_validate_int = Y[half:three_fourths]
y_test_int = Y[three_fourths:]

# convert labels to one-hot encoding
y_train = np_utils.to_categorical(y_train_int-1)
y_validate = np_utils.to_categorical(y_validate_int-1)
y_test = np_utils.to_categorical(y_test_int-1)

# convert data to NHWC (trials, channels, samples, kernels) format
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], kernels)
X_validate = X_validate.reshape(X_validate.shape[0], X_validate.shape[1], X_validate.shape[2], kernels)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], kernels)

print('x_train shape: ', X_train.shape, '\ny_train shape: ', y_train.shape)
################################################################
## Call EEGNet

model = EEGNet(nb_classes=num_labels, Chans=X_train.shape[1], Samples=X_train.shape[2],
               dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16,
                 dropoutType= 'Dropout')

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

###############################################################################
# if the classification task was imbalanced (significantly more trials in one
# class versus the others) you can assign a weight to each class during 
# optimization to balance it out. This data is approximately balanced so we 
# don't need to do this, but is shown here for illustration/completeness. 
###############################################################################

# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
class_weights = {0:1, 1:1, 2:1, 3:1, 4:1}



x_train shape:  (90, 64, 656, 1) 
y_train shape:  (90, 5)


2023-10-02 19:35:57.862028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-02 19:35:57.862068: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-02 19:35:57.862096: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kaleb-School): /proc/driver/nvidia/version does not exist
2023-10-02 19:35:57.862455: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


test print statement








In [4]:
################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################
fittedModel = model.fit(X_train, y_train, batch_size = 16, epochs = 30, 
                        verbose = 2, validation_data=(X_validate, y_validate),
                        callbacks=[checkpointer], class_weight = class_weights)

# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

###############################################################################
# can alternatively used the weights provided in the repo. If so it should get
# you 93% accuracy. Change the WEIGHTS_PATH variable to wherever it is on your
# system.
###############################################################################

# WEIGHTS_PATH = /path/to/EEGNet-8-2-weights.h5 
# model.load_weights(WEIGHTS_PATH)

###############################################################################
# make prediction on test set.
###############################################################################

probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

Epoch 1/30

Epoch 1: val_loss improved from inf to 1.57136, saving model to /tmp/checkpoint.h5
6/6 - 4s - loss: 1.7437 - accuracy: 0.2444 - val_loss: 1.5714 - val_accuracy: 0.4222 - 4s/epoch - 622ms/step
Epoch 2/30

Epoch 2: val_loss improved from 1.57136 to 1.54208, saving model to /tmp/checkpoint.h5
6/6 - 2s - loss: 1.4658 - accuracy: 0.5111 - val_loss: 1.5421 - val_accuracy: 0.5111 - 2s/epoch - 321ms/step
Epoch 3/30

Epoch 3: val_loss improved from 1.54208 to 1.51467, saving model to /tmp/checkpoint.h5
6/6 - 2s - loss: 1.3487 - accuracy: 0.5778 - val_loss: 1.5147 - val_accuracy: 0.4889 - 2s/epoch - 344ms/step
Epoch 4/30

Epoch 4: val_loss improved from 1.51467 to 1.48712, saving model to /tmp/checkpoint.h5
6/6 - 2s - loss: 1.2569 - accuracy: 0.6222 - val_loss: 1.4871 - val_accuracy: 0.5111 - 2s/epoch - 331ms/step
Epoch 5/30

Epoch 5: val_loss improved from 1.48712 to 1.47150, saving model to /tmp/checkpoint.h5
6/6 - 2s - loss: 1.1863 - accuracy: 0.6222 - val_loss: 1.4715 - val_accur

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance

# evaluate the model
probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Baseline accuracy: %f " % (acc))

print("x_test.shape: ", X_test.shape)
print("y_test.shape: ", y_test.shape)




2/2 [==============================] - 0s 42ms/step
Baseline accuracy: 0.622222 
x_test.shape:  (45, 64, 656, 1)
y_test.shape:  (45, 5)


In [6]:
int_labels_y_test = np.argmax(model.predict(X_test), axis=1)
print(int_labels_y_test.shape)

2/2 [==============================] - 0s 77ms/step
(45,)


In [7]:
# permutation importance
result = permutation_importance(model, X_test, y_test, n_repeats=30, random_state=42)

# get indices of the most important features
importances = result.importances_mean
important_electrodes = result.importances_mean.argsort()[::-1][:16]
print("16 important electrodes: ", important_electrodes)

ValueError: Found array with dim 4. None expected <= 2.

In [21]:
#Gpt edit: 
X_2d = np.mean(X, axis=2)
y_2d = Y

# # split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_2d, y_2d, test_size=0.2, random_state=42)



In [22]:
fittedModel = model.fit(X_train, y_train)

ValueError: in user code:

    File "/home/kaleb/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/kaleb/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/kaleb/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/kaleb/.local/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/kaleb/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/kaleb/.local/lib/python3.10/site-packages/keras/engine/input_spec.py", line 250, in assert_input_compatibility
        raise ValueErr